Node provides a wonderful way to compile and run code within V8 Virtual Machine contexts, via the `vm` module:

In [1]:
var vm = require('vm')

In [2]:
vm

{ Script: [Function: ContextifyScript],
  createScript: [Function],
  createContext: [Function],
  runInDebugContext: [Function],
  runInContext: [Function],
  runInNewContext: [Function],
  runInThisContext: [Function],
  isContext: [Function: isContext] }

The important API for keeping a running session with node is `createContext`. It lets you create a sandbox which can be provided a global context, providing modules and functions people expect like `console.log`, `setTimeout`, and `require`. Below we'll create a "sandbox" that isn't much of one since it will get the current globals, including require.

In [3]:
var sandbox = vm.createContext(
    Object.assign({}, global)
)

In [4]:
sandbox.console

Console {
  log: [Function: bound ],
  info: [Function: bound ],
  warn: [Function: bound ],
  error: [Function: bound ],
  dir: [Function: bound ],
  time: [Function: bound ],
  timeEnd: [Function: bound ],
  trace: [Function: bound trace],
  assert: [Function: bound ],
  Console: [Function: Console] }

To run in the sandbox we use `vm.runInContext` which has this signature

```js
runInContext(code: string, sandbox: context): any
```

In [5]:
var result = vm.runInContext("var b = \{ a: 2, require \}; b", sandbox)

In [6]:
result.a

2

## Creating a display function for a nodejs kernel

In order to be able to asynchronously write back to the frontend, we'll want a `display` function.

In [10]:
var hookedDisplay = console.log.bind(console) // Pretend this is a rich display

var sandbox = vm.createContext(
  Object.assign(
    { display: hookedDisplay },
    global
  )
)

In [23]:
// code comes in on an `execute_request`
code = "display('hey')"
// { execute_request: { code: "display('hey')" }}

// We execute the code
result = vm.runInContext(code, sandbox)

// Responses:
//   { display_data: { data: 'text/plain': 'hey' }}
//   { execute_result: { data: 'text/plain': 'undefined' }}


hey
